In [1]:
# Simulating dynamics of Argon atoms
# Starting with outlining general kinematics of particles

Week 1:
Play around with this system. Start by simulating the time evolution of a few particles in a periodic box, 
add the forces due to the Lennard-Jones potential. Check how the total energy of your system evolves over time.
It's easier to start with a 2D system, but plan to switch to 3D at a later stage.
Week 2:
Derive the expression of the kinetic energy in dimensionless units
Write a molecular dynamics code that uses dimensionless units and simulate a few atoms. 
Plot both kinetic, potential and total energy.
Boundary condition, minimal image convention

In [5]:
import numpy as np
from astropy import constants as const
from astropy import units as u

In [28]:
# Lennard-Jones potential 
def LJP(r):
    '''Lennard-Jones potential formula
    Describes the potential of the system given a distance between two particles
    in dimensionless units, no sigma or epsilon
    function of magnitude distance r'''
    return 4*(1/(r**12.) - 1/(r**6.))

def dUdr(r):
    ''' Derivative of LJP WRT r
    in dimensionless units, no sigma or epsilon
    function of magnitude distance r'''
    return -48./(r**13.) + 24/(r**7.)

def acceleration(derivU, x, r):
    '''Force from LJP potential
    derivU from function dUdr
    x is vector (dimensionless)
    r is magnitude (dimensionless)'''
    nablaU = derivU*(x/r)
    return -nablaU

# New velocity
def next_velocity(v_prev, a, t):
    '''Finding the next velocity value
    previous velocity v_prev
    acceleration a at x_prev
    timestep t
    Dimensionless units'''
    return v_prev + a*t

# New position given a timestep
def next_position(x_prev, v_prev, t):
    '''The next position value
    previous position x_prev
    previous velocity v_prev
    timestep t
    Dimensionless units'''
    return x_prev + v_prev*t

In [29]:
'''Parameters given in dimensionless units of sigma
 Perhaps convert between units later'''
# Variables
# Argon
# epsilon = const.k_B * 119.8*u.K
# sigma = 3.405*u.AA
# mass = 39.948*u.u

'Parameters given in dimensionless units of sigma\n Perhaps convert between units later'

In [51]:
# The force at x is the negative gradient of the potential at x
# The gradient of the potential is the derivative of the potential WRT r times x/r
# where r is the magnitude of the distance and x is a vector

# Initial conditions
h = 0.05 # time step
num_time = 10  # number of time steps
num_part = 2

x_i_init = np.array([10., 0., 0.]) # initial position vector of particle i
v_i_init = np.array([1.0, 0., 0.]) # initial velocity vector of particle i

x_j_init = np.array([0., 0., 0.])
v_j_init = np.array([-1.0, 0., 0.])

r_init = np.sqrt((x_i_init[0] - x_j_init[0])**2. + (x_i_init[1] - x_j_init[1])**2. + 
                   (x_i_init[2] - x_j_init[2])**2.) # magnitude

acc_i = acceleration(dUdr(r_init), x_i_init, r_init) # acceleration of particle i
acc_j = acceleration(dUdr(r_init), x_j_init, r_init) # acceleration of particle j
print(acc_i)
print(acc_j)

[-2.3999952e-06 -0.0000000e+00 -0.0000000e+00]
[-0. -0. -0.]


In [47]:
#Test
# next_v = next_velocity(v_init, acc, h)
# print(next_v)
# next_x = next_position(x_init, v_init, h)
# print(next_x)

In [49]:
timesteps = np.zeros((num_part, num_time, 6))  # 6 degrees of freedom: 3 position coordinates, 3 velocities
print(timesteps)
timesteps[0, :3] = x_init
timesteps[0, 3:] = v_init
# print(timesteps)
# Change in position adn velocity
for t in range(1, num_time):
    v_next = next_velocity(timesteps[t-1, 3:], acc, h)
    x_next = next_position(timesteps[t-1, :3], timesteps[t-1, 3:], h)
    timesteps[t, :3] = x_next
    timesteps[t, 3:] = v_next

print(timesteps)

[[[0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]]]


ValueError: could not broadcast input array from shape (3) into shape (3,6)